In [ ]:
# Import module
import xgboost as xgb
import pandas as pd
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

# Kerasに付属の手書き数字画像データをダウンロード
np.random.seed(0)
(X_train_base, labels_train_base), (X_test, labels_test) = mnist.load_data()

# Training set を学習データ（X_train, labels_train）と検証データ（X_validation, labels_validation）に8:2で分割する
X_train,X_validation,labels_train,labels_validation = train_test_split(X_train_base,labels_train_base,test_size = 0.2)

# 各画像は行列なので1次元に変換→X_train,X_validation,X_testを上書き
X_train = X_train.reshape(-1,784)
X_validation = X_validation.reshape(-1,784)
X_test = X_test.reshape(-1,784)

#正規化
X_train = X_train.astype('float32')
X_validation = X_validation.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_validation /= 255
X_test /= 255

# 訓練・テストデータの設定
train_data = xgb.DMatrix(X_train, label=labels_train)
eval_data = xgb.DMatrix(X_validation, label=labels_validation)
X_data = xgb.DMatrix(X_test, label=labels_test)

#経過時間計測
import time
start = time.time()

xgb_params = {
    # 多値分類問題
    'objective':'multi:softmax',
    # クラス数
    'num_class': 10,
    # 学習用の指標 (Multiclass logloss)
    'eval_metric': 'mlogloss',
    }
evals = [(train_data, 'train'), (eval_data, 'eval')]

gbm = xgb.train(
    xgb_params,
    train_data,
    num_boost_round=100,
    early_stopping_rounds=10,
    evals=evals,
    )

preds = gbm.predict(X_data)
from sklearn.metrics import accuracy_score
print('accuracy_score:{}'.format(accuracy_score(labels_test, preds)))

#経過時間
print('elapsed_timetime:{}'.format(time.time()-start))

11493376/11490434 [==============================] - 7s 1us/step
[0]	train-mlogloss:1.36145	eval-mlogloss:1.37945
[1]	train-mlogloss:1.02845	eval-mlogloss:1.05507
[2]	train-mlogloss:0.80982	eval-mlogloss:0.84405
[3]	train-mlogloss:0.65517	eval-mlogloss:0.69440
[4]	train-mlogloss:0.54093	eval-mlogloss:0.58534
[5]	train-mlogloss:0.45045	eval-mlogloss:0.49992
[6]	train-mlogloss:0.37916	eval-mlogloss:0.43248
[7]	train-mlogloss:0.32197	eval-mlogloss:0.37777
[8]	train-mlogloss:0.27675	eval-mlogloss:0.33486
[9]	train-mlogloss:0.24008	eval-mlogloss:0.30103
[10]	train-mlogloss:0.21141	eval-mlogloss:0.27420
